In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
telecom_df = pd.read_csv('filtered.csv')

### PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(svd_solver='randomized', random_state=42)
pca.fit(X_train)
pca.components_

fig = plt.figure(figsize = (8,8))
plt.scatter(pcs_df.PC1, pcs_df.PC2)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
for i, txt in enumerate(pcs_df.Feature):
    plt.annotate(txt, (pcs_df.PC1[i],pcs_df.PC2[i]))
plt.tight_layout()
plt.show()

In [ ]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 4)
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]}]
model = SVC(kernel="rbf")
model_cv = GridSearchCV(estimator = model, param_grid = hyper_params, scoring= 'accuracy', 
                        cv = folds, verbose = 1,return_train_score=True)      
model_cv.fit(X_train, y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
best_params = {"C": 100, "gamma": 0.0001, "kernel":"rbf"}

# model
model = SVC(C=100, gamma=0.0001, kernel="rbf")

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# metrics
print(metrics.confusion_matrix(y_test, y_pred), "\n")
print("accuracy", metrics.accuracy_score(y_test, y_pred))
print("precision", metrics.precision_score(y_test, y_pred))
print("sensitivity/recall", metrics.recall_score(y_test, y_pred))